In [1]:
%reload_ext autoreload
%autoreload 2

import realsense_photo as rsp
import CustomDataset as cdset
import pickle


from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
#loading architexture

import segmentation_models_pytorch as smp
learning_rate = 0.001

#creating new model
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=4,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
#or loading already trained model
#model = torch.load("model")

criterion = smp.losses.JaccardLoss('multiclass', classes=None, log_loss=False, from_logits=True, smooth=0.0, eps=1e-07)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print("model load success")

model load success


In [22]:
#loading some series of photos
series = 0
pictures, labels = rsp.getDataVector("data/home", series)
print("pictures shape ", pictures.shape)
print("labels shape", labels.shape)
train_loader, val_loader = rsp.makeLoader(pictures, labels)
print("loaded ", pictures.shape[0]," photos in ",series, " series")

pictures shape  (7, 480, 640, 4)
labels shape (7, 480, 640)
loaded  7  photos in  0  series


In [13]:
#gym training

num_epochs = 4

total_step = len(train_loader)
loss_list = []
acc_list = []
model = model.float()
#model.to(device)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Прямой запуск
        #images = images.to(device)
        #labels = labels.to(device)
        
        model.train()
        outputs = model(images.float())
        loss = criterion(outputs, labels.long())
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/4], Step [1/1], Loss: 0.5698
Epoch [2/4], Step [1/1], Loss: 0.5657
Epoch [3/4], Step [1/1], Loss: 0.5567


KeyboardInterrupt: 

In [23]:
targetPhoto = 0
#model.to(torch.device("cpu"))
for i, (images, labels) in enumerate(val_loader):
        if (i == targetPhoto):
            print("loading ", i, " photo")
            outputs = model(images.float())
            loss = criterion(outputs, labels.long())
            print('Val Loss: {:.4f}'
                      .format(loss.item()))
            
            pred = outputs.log_softmax(dim=1).exp()
            picture = np.zeros(1*1*480*640)
            picture = picture.reshape((480,640))
            
            for x in range(480):
                for y in range(640):
                    maxim = 0
                    maxID = 0
                    for p in range(7):
                        if (maxim<pred[0][p][x][y]):
                            maxim = pred[0][p][x][y]
                            maxID = p
                    picture[x][y] = maxID
                   
                   
            #rgb = images[:,0:3,:,:]
            rgb_real = np.zeros(480*640*3)
            rgb_real = rgb_real.reshape((480,640,3))
            
            for x in range(480):
                for y in range(640):
                    rgb_real[x][y][0] = images[0][0][x][y]
                    rgb_real[x][y][1] = images[0][1][x][y]
                    rgb_real[x][y][2] = images[0][2][x][y]
            
            rsp.drawLabelMap(rgb_real, picture)
        

loading  0  photo
Val Loss: 0.4943
(480, 640, 3)
(480, 640, 3)


In [ ]:
#file = open("model.pickleobj", "wb")
#pickle.dump(model, file)
#file.close()

#torch.save("model", model)